In [1]:
from policyengine_core.model_api import *
from policyengine_core.charts import get_api_chart_data
import pandas as pd
import json

per_child_phase_in = {
    "gov.contrib.congress.wyden_smith.per_child_actc_phase_in": {
        "year:2023:3": True,
    },
}
lookback = {
    "gov.contrib.congress.wyden_smith.actc_lookback": {
        "year:2024:2": True,
    },
}
refundable_increase = {
    "gov.irs.credits.ctc.refundable.individual_max": {
        "year:2023:1": 1_800,
        "year:2024:1": 1_900,
        "year:2025:1": 2_100,
    },
}
indexing_ctc = {
    "gov.irs.credits.ctc.amount.base[0].amount": {
        "year:2024:2": 2_100,
    },
}

reforms = [per_child_phase_in, lookback, refundable_increase, indexing_ctc]

provisions = [
    "Per-child phase-in",
    "Lookback",
    "Refundable increase",
    "Indexing CTC",
]

def get_outputs(api_id: int, year: int) -> dict:
    budget_data = get_api_chart_data("us", api_id, "budget", "enhanced_us", year)
    return {
        "Budget": budget_data["budgetary_impact"],
        "PolicyEngine": f"[#{api_id}](https://policyengine.org/us/policy?reform={api_id}&baseline=2&time_period={year}&region=enhanced_us&focus=policyOutput.policyBreakdown)",
    }

def get_per_provision_table(reforms: list, provisions: list, year: int) -> pd.DataFrame:
    table_data = []
    for i in range(1, len(reforms) + 1):
        print(year, provisions[i-1])
        sliced_reform = {}
        for reform in reforms[:i]:
            sliced_reform.update(reform)
        api_id = Reform.from_dict(sliced_reform, "us").api_id
        outputs = get_outputs(api_id, year)
        table_data.append({
            "Provision": provisions[i - 1],
            "Cost": outputs["Budget"],
            "Link": outputs["PolicyEngine"],
            "Year": year,
        })
    
    combined_row = json.loads(json.dumps(table_data[-1]))
    combined_row["Provision"] = "Combined"
    table_data.append(combined_row)

    df = pd.DataFrame(table_data)
    df["Cost"][1:-1] = df["Cost"][1:-1].values - df["Cost"][:-2].values

    return df

def get_multi_year_provision_table(reforms, provisions, years):
    dfs = []
    for year in years:
        df = get_per_provision_table(reforms, provisions, year)
        dfs.append(df)
    return pd.concat(dfs, ignore_index=True)

df = get_multi_year_provision_table(reforms, provisions, [2023, 2024, 2025])
df["Cost"] = df["Cost"].apply(lambda x: float(f"{x/1e9:.1f}"))

2023 Per-child phase-in
2023 Lookback
2023 Refundable increase
2023 Indexing CTC


/var/folders/8f/pgfhysmd5ls3jnxb_5j7yy340000gn/T/ipykernel_61012/2791506433.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Cost"][1:-1] = df["Cost"][1:-1].values - df["Cost"][:-2].values


2024 Per-child phase-in
2024 Lookback
2024 Refundable increase
2024 Indexing CTC


/var/folders/8f/pgfhysmd5ls3jnxb_5j7yy340000gn/T/ipykernel_61012/2791506433.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Cost"][1:-1] = df["Cost"][1:-1].values - df["Cost"][:-2].values


2025 Per-child phase-in
2025 Lookback
2025 Refundable increase
2025 Indexing CTC


/var/folders/8f/pgfhysmd5ls3jnxb_5j7yy340000gn/T/ipykernel_61012/2791506433.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Cost"][1:-1] = df["Cost"][1:-1].values - df["Cost"][:-2].values


In [2]:
linked_values = []
for i in range(len(df)):
    link = df["Link"][i]
    link = link[9:-1]
    cost = df["Cost"][i]
    cost = cost if cost == 0 else -cost
    linked_values.append(f"[{cost}]({link})")

df["Value"] = linked_values

pivot_df = df.pivot_table(index="Provision", columns="Year", values="Value", aggfunc="first").fillna("")
# Order of provisions is Per-child phase-in, Lookback, Refundable increase, Indexing CTC
pivot_df = pivot_df.reindex(["Per-child phase-in", "Lookback", "Refundable increase", "Indexing CTC", "Combined"])
pivot_df = pivot_df.reset_index().rename_axis(None, axis=1).set_index("Provision", drop=True)

pivot_df.rename(columns={2023: "2023 (PE)", 2024: "2024 (PE)", 2025: "2025 (PE)"}, inplace=True)
pivot_df["2023 (JCT)"] = [5.5, 0, 2.7, 0, 8.2]
pivot_df["2024 (JCT)"] = [6.0, 0.8, 2.5, 1.4, 10.7]
pivot_df["2025 (JCT)"] = [5.7, 0.7, 3.4, 4.7, 14.7]

def flt(link):
    return float(link[1:link.index("]")])

pivot_df["2023-2025 (JCT)"] = pivot_df["2023 (JCT)"] + pivot_df["2024 (JCT)"] + pivot_df["2025 (JCT)"]
pivot_df["2023-2025 (PE)"] = pivot_df["2023 (PE)"].apply(flt) + pivot_df["2024 (PE)"].apply(flt) + pivot_df["2025 (PE)"].apply(flt)


NameError: name 'Markdown' is not defined

In [4]:
from IPython.display import Markdown
pivot_df = pivot_df.reindex(columns=["2023 (JCT)", "2023 (PE)", "2024 (JCT)", "2024 (PE)", "2025 (JCT)", "2025 (PE)", "2023-2025 (JCT)", "2023-2025 (PE)"])

Markdown(pivot_df.to_markdown())

| Provision           |   2023 (JCT) | 2023 (PE)                                                                                                                                |   2024 (JCT) | 2024 (PE)                                                                                                                                 |   2025 (JCT) | 2025 (PE)                                                                                                                                 |   2023-2025 (JCT) |   2023-2025 (PE) |
|:--------------------|-------------:|:-----------------------------------------------------------------------------------------------------------------------------------------|-------------:|:------------------------------------------------------------------------------------------------------------------------------------------|-------------:|:------------------------------------------------------------------------------------------------------------------------------------------|------------------:|-----------------:|
| Per-child phase-in  |          5.5 | [3.8](https://policyengine.org/us/policy?reform=49823&baseline=2&time_period=2023&region=enhanced_us&focus=policyOutput.policyBreakdown) |          6   | [4.5](https://policyengine.org/us/policy?reform=49823&baseline=2&time_period=2024&region=enhanced_us&focus=policyOutput.policyBreakdown)  |          5.7 | [4.3](https://policyengine.org/us/policy?reform=49823&baseline=2&time_period=2025&region=enhanced_us&focus=policyOutput.policyBreakdown)  |              17.2 |             12.6 |
| Lookback            |          0   | [0.0](https://policyengine.org/us/policy?reform=49855&baseline=2&time_period=2023&region=enhanced_us&focus=policyOutput.policyBreakdown) |          0.8 | [3.2](https://policyengine.org/us/policy?reform=49855&baseline=2&time_period=2024&region=enhanced_us&focus=policyOutput.policyBreakdown)  |          0.7 | [3.8](https://policyengine.org/us/policy?reform=49855&baseline=2&time_period=2025&region=enhanced_us&focus=policyOutput.policyBreakdown)  |               1.5 |              7   |
| Refundable increase |          2.7 | [1.5](https://policyengine.org/us/policy?reform=49861&baseline=2&time_period=2023&region=enhanced_us&focus=policyOutput.policyBreakdown) |          2.5 | [2.0](https://policyengine.org/us/policy?reform=49861&baseline=2&time_period=2024&region=enhanced_us&focus=policyOutput.policyBreakdown)  |          3.4 | [2.9](https://policyengine.org/us/policy?reform=49861&baseline=2&time_period=2025&region=enhanced_us&focus=policyOutput.policyBreakdown)  |               8.6 |              6.4 |
| Indexing CTC        |          0   | [0.0](https://policyengine.org/us/policy?reform=49862&baseline=2&time_period=2023&region=enhanced_us&focus=policyOutput.policyBreakdown) |          1.4 | [4.8](https://policyengine.org/us/policy?reform=49862&baseline=2&time_period=2024&region=enhanced_us&focus=policyOutput.policyBreakdown)  |          4.7 | [5.7](https://policyengine.org/us/policy?reform=49862&baseline=2&time_period=2025&region=enhanced_us&focus=policyOutput.policyBreakdown)  |               6.1 |             10.5 |
| Combined            |          8.2 | [5.3](https://policyengine.org/us/policy?reform=49862&baseline=2&time_period=2023&region=enhanced_us&focus=policyOutput.policyBreakdown) |         10.7 | [14.5](https://policyengine.org/us/policy?reform=49862&baseline=2&time_period=2024&region=enhanced_us&focus=policyOutput.policyBreakdown) |         14.7 | [16.7](https://policyengine.org/us/policy?reform=49862&baseline=2&time_period=2025&region=enhanced_us&focus=policyOutput.policyBreakdown) |              33.6 |             36.5 |